<a href="https://colab.research.google.com/github/a2kiti/colab_web_demo/blob/master/CGAN_web_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#Connecting to googledrive
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
modelpath = 'gdrive/My Drive/Colab Notebooks/CGAN_model.hdf5'

In [0]:
from keras.models import load_model
import numpy as np
import os
from scipy.misc import imsave

In [0]:
gen = load_model(modelpath)
z_dim = 100
class_num = 10

#Executing once to avoid errors
_ = gen.predict(np.zeros((1, z_dim+class_num)))

In [0]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

In [0]:
#Writing to html file on colab

html = """

<!DOCTYPE html>
<html>
<head>
<meta charset="utf-8" />
</head>
<body>

<form action="/post" method="post">
<p>label：<input type="text" name="label"></p>
<p><input type="submit" value="send"/></p>
</form>

{% if ulr_Image %}

    <p>Generated image</p>
    <p><img src="{{ ulr_Image }}"></p>
    

{% endif %}

</body>
</html>

"""

#Saving html file
if not os.path.isdir( "templates" ):
    os.makedirs( "templates" )
with open("templates/index.html", mode='w') as f:
    f.write(html)

#Making tmp folder to use temporarily stored
if not os.path.isdir( "tmp" ):
    os.makedirs( "tmp" )

In [0]:
get_ipython().system_raw('./ngrok http 6006 &')
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

from flask import Flask, render_template, send_from_directory, request, redirect, url_for

app = Flask(__name__)

@app.route("/")
def index():
    return render_template( "index.html" )

@app.route('/post', methods=['POST'])
def post():
    label = request.form['label']
    if label == "0" or label == "1" or label == "2" or label == "3" or label == "4" or label == "5" or label == "6" or label == "7" or label == "8" or label == "9": 
        label = int(label)
    else:
        return redirect(url_for('index'))
    
    noise = np.random.normal(0, 1, (1, z_dim))
    noise_label = np.zeros((1, class_num))
    noise_label[:, label] = 1
    noise_with_label = np.concatenate((noise, noise_label),axis=1)

    img = (0.5 * gen.predict(noise_with_label) + 0.5)
    #mono or color
    if img.shape[-1] == 1:
        img=img.reshape(img.shape[1],img.shape[2])
    else:
        img = img.reshape(img.shape[1],img.shape[2],img.shape[3])
    fileName = str(np.random.randint(1000)) + ".jpg"
    imsave("tmp/" + fileName, img)
    ulr_Image = "tmp/" + fileName
    
    return render_template( "index.html",
                                        ulr_Image = ulr_Image)

@app.route('/tmp/<filename>')
def uploaded_file(filename):
    return send_from_directory("./tmp", filename)

if __name__ == '__main__':
    app.run(port=6006)